In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# Define the Generator
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )

    def forward(self, z):
        return self.model(z)

# Define the Discriminator
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Hyperparameters
input_dim = 1
noise_dim = 1
lr = 0.0002
batch_size = 64
epochs = 10000

def true_function(x):
    epsilon = torch.randn_like(x) * 0.1  # Add noise
    return -x + epsilon

# Initialize Generator and Discriminator
generator = Generator(noise_dim, input_dim)
discriminator = Discriminator(input_dim)

# Loss and Optimizers
criterion = nn.BCELoss()
g_optimizer = optim.Adam(generator.parameters(), lr=lr)
d_optimizer = optim.Adam(discriminator.parameters(), lr=lr)

# Training loop
for epoch in range(epochs):
    # Generate real data
    real_data = torch.rand(batch_size, input_dim) * 2 - 1  # Uniform distribution in [-1, 1]
    real_labels = torch.ones(batch_size, 1)  # Label for real data
    fake_labels = torch.zeros(batch_size, 1)  # Label for fake data

    # Train Discriminator
    d_optimizer.zero_grad()
    real_outputs = discriminator(real_data)
    real_loss = criterion(real_outputs, real_labels)

    noise = torch.randn(batch_size, noise_dim)
    fake_data = generator(noise)
    fake_outputs = discriminator(fake_data.detach())
    fake_loss = criterion(fake_outputs, fake_labels)

    d_loss = real_loss + fake_loss
    d_loss.backward()
    d_optimizer.step()

    # Train Generator
    g_optimizer.zero_grad()
    fake_outputs = discriminator(fake_data)
    g_loss = criterion(fake_outputs, real_labels)  # Try to fool the discriminator
    g_loss.backward()
    g_optimizer.step()

    # Print progress
    if (epoch + 1) % 1000 == 0:
        print(f"Epoch [{epoch + 1}/{epochs}], D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}")

# Test the Generator
noise = torch.randn(10, noise_dim)
with torch.no_grad():
    generated_data = generator(noise)
print("Generated Data:", generated_data)


Epoch [1000/10000], D Loss: 1.3440, G Loss: 0.7489
Epoch [2000/10000], D Loss: 1.1951, G Loss: 0.8892
Epoch [3000/10000], D Loss: 1.1937, G Loss: 0.8330
Epoch [4000/10000], D Loss: 1.2375, G Loss: 0.8077
Epoch [5000/10000], D Loss: 1.2757, G Loss: 0.7988
Epoch [6000/10000], D Loss: 1.4561, G Loss: 0.7116
Epoch [7000/10000], D Loss: 1.2862, G Loss: 0.7629
Epoch [8000/10000], D Loss: 1.5045, G Loss: 0.5712
Epoch [9000/10000], D Loss: 1.2991, G Loss: 0.8159
Epoch [10000/10000], D Loss: 1.4321, G Loss: 0.5589
Generated Data: tensor([[0.5042],
        [0.5582],
        [0.7426],
        [0.7180],
        [0.4550],
        [0.4462],
        [0.7691],
        [1.0087],
        [0.5869],
        [0.4509]])


In [22]:

noise = torch.randn(10, noise_dim)*5+10
with torch.no_grad():
    generated_data = generator(noise)
print("Generated Data:", generated_data)

Generated Data: tensor([[6.6861],
        [5.6183],
        [5.0435],
        [2.8878],
        [2.4957],
        [5.5257],
        [4.9065],
        [2.6250],
        [1.0314],
        [4.1515]])
